In [1]:
import AFQ.data as afqd
import cloudknot as ck
import importlib
import s3fs

/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dipy/stats/__init__.py:7: UserWarning: The `dipy.stats` module is still under heavy development and functionality, as well as the API is likely to change in future versions of the software
  warnings.warn(w_string)
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials f

In [2]:
importlib.reload(afqd)

<module 'AFQ.data' from '/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/AFQ/data.py'>

In [3]:
ixi = afqd.S3BIDSStudy(
    study_id='ixi', bucket='ixi.bids', s3_prefix='', subjects=5,
    anon=False, use_participants_tsv=False, random_seed=42
)

Retrieving subject S3 keys
[########################################] | 100% Completed |  1.0s


In [4]:
ixi.non_sub_s3_keys

{'raw': ['ixi.bids/dataset_description.json',
  'ixi.bids/dwi.bval',
  'ixi.bids/dwi.bvec'],
 'derivatives': []}

In [5]:
ixi.download('./ixi-bids')

In [6]:
import os.path as op

fs = s3fs.S3FileSystem()

for fn in ixi.non_sub_s3_keys['raw']:
    fs.get(fn, op.join('.', 'ixi-bids', op.basename(fn)))

In [7]:
s0 = ixi.subjects[0]
s1 = ixi.subjects[1]

In [8]:
import os.path as op

op.commonpath(list(s0.files['raw'].values()) + list(s1.files['raw'].values()))

'/Users/richford/projects/neuro/hbn-hub/notebooks/ixi-bids'

In [9]:
def preprocess_ixi_subject(subject_id):
    import AFQ.data as afqd
    import os
    import subprocess
    from s3fs import S3FileSystem
    
    ixi = afqd.afqd.S3BIDSStudy(
        study_id='ixi', bucket='ixi.bids', s3_prefix='', subjects=subject_id,
        anon=False, use_participants_tsv=False
    )
        
    ixi.download("./ixi")
    
    fs = s3fs.S3FileSystem()
        
    for fn in ixi.non_sub_s3_keys['raw']:
        if 'bval' in fn or 'bvec' in fn:
            fs.get(fn, os.path.join('.', 'ixi-bids', op.basename(fn)))
    
    subprocess.run(
        ["qsiprep",
         "./ixi",
         "./ixi-preproc",
         subject_id],
        shell=True,
        check=True,
    )

    def upload_dir(path):
        output_dir = 'ixi.bids/derivatives/qsiprep_ck'
        if fs.exists(output_dir):
            walk_dir = os.walk(path)
            _, dirs, files = next(walk_dir)
            for fn in dirs + files:
                fs.put(os.path.join(path, fn), output_dir, recursive=True)
        else:
            fs.put(path, output_dir, recursive=True)
            
    upload_dir("./ixi-preproc")

In [10]:
di = ck.DockerImage(
    func=preprocess_ixi_subject,
    base_image="pennbbl/qsiprep",
    github_installs="https://github.com/richford/pyAFQ.git@s3-bids-fetch",
)

In [11]:
di.build_path

'/Users/richford/projects/neuro/hbn-hub/notebooks/cloudknot_docker_preprocess-ixi-subject_t1r5mhji'

## Instruction

Go to that directory and edit the Dockerfile to include `--ignore-installed` to the `pip install` commands.

In [12]:
di.build(tags=["ixi_preproc_20200708"])

In [20]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [14]:
di.images

[{'name': 'cloudknot/preprocess-ixi-subject', 'tag': 'ixi_preproc_20200708'}]

In [15]:
di.repo_uri

In [16]:
ck.get_ecr_repo()

DEBUG:urllib3.util.retry:Converted retries value: False -> Retry(total=False, connect=None, read=None, redirect=0, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.ecr.us-east-2.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://api.ecr.us-east-2.amazonaws.com:443 "POST / HTTP/1.1" 200 331


'cloudknot'

In [17]:
repo = ck.aws.DockerRepo(name=ck.get_ecr_repo())

DEBUG:urllib3.util.retry:Converted retries value: False -> Retry(total=False, connect=None, read=None, redirect=0, status=None)
DEBUG:urllib3.connectionpool:https://api.ecr.us-east-2.amazonaws.com:443 "POST / HTTP/1.1" 200 331
DEBUG:urllib3.util.retry:Converted retries value: False -> Retry(total=False, connect=None, read=None, redirect=0, status=None)
DEBUG:urllib3.connectionpool:https://api.ecr.us-east-2.amazonaws.com:443 "POST / HTTP/1.1" 200 331


In [18]:
repo.repo_uri

'454929164628.dkr.ecr.us-east-2.amazonaws.com/cloudknot'

In [19]:
# This command takes a few hours because the docker image is large
di.push(repo=repo)

DEBUG:docker.utils.config:Trying paths: ['/Users/richford/.docker/config.json', '/Users/richford/.dockercfg']
DEBUG:docker.utils.config:Found file at path: /Users/richford/.docker/config.json
DEBUG:docker.auth:Found 'auths' section
DEBUG:docker.auth:Auth data for 454929164628.dkr.ecr.us-east-1.amazonaws.com is absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for 454929164628.dkr.ecr.us-east-2.amazonaws.com is absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for 454929164628.dkr.ecr.us-west-2.amazonaws.com is absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for 455598791984.dkr.ecr.us-west-2.amazonaws.com is absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for https://455598791984.dkr.ecr.us-west-2.amazonaws.com is absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Found 'credsStore' section
DEBUG:docker.utils.c

In [21]:
knot = ck.Knot(
    name="ixi_preproc_20200708_arh_v2",
    docker_image=di,
    pars_policies=('AmazonS3FullAccess',),
    resource_type="SPOT",
    bid_percentage=100,
    memory=64000,
    job_def_vcpus=32,
)

In [22]:
sub_ids = [s.subject_id for s in ixi.subjects]
sub_ids

['sub-IXI108', 'sub-IXI538', 'sub-IXI039', 'sub-IXI139', 'sub-IXI631']

In [23]:
result_futures = knot.map(sub_ids)

In [25]:
knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
52b26af0-1dc8-4096-b4df-73eb17405be5        ixi-preproc-20200708-arh-v2-0        PENDING  


In [ ]:
print([j.status for j in knot.jobs])

In [ ]:
knot.clobber(clobber_pars=True)